In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from data_helpers import clean_str
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import make_pipeline as make_pipeline_imb
# from imblearn.metrics import classification_report_imbalanced
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import pandas as pd

In [ ]:
#seg_types = ['seg_tlex','seg_sylSeg','seg_tcc','seg_icu']

seg = 'seg_lextoplus'
directory = 'Dataset/'
positive_file = directory +  'Train/all_clickbait_' + seg + '_Train'
negative_file = directory + 'Train/all_non_clickbait_' + seg + '_Train'

test_positive_file = directory+'Test/all_clickbait_'+seg+'_Test'
test_negative_file = directory+'Test/all_non_clickbait_'+seg+'_Test'

In [ ]:
positive_examples = list(open(positive_file ,encoding="utf8").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_file ,encoding="utf8").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
x_text = [clean_str(sent) for sent in x_text]
x_text = [s.split("|") for s in x_text]
# Generate labels

positive_labels = [1 for _ in positive_examples]
negative_labels = [0 for _ in negative_examples]
y = np.concatenate([positive_labels, negative_labels], 0)

In [ ]:
test_positive_examples = list(open(test_positive_file ,encoding="utf8").readlines())
test_positive_examples = [s.strip() for s in test_positive_examples]
test_negative_examples = list(open(test_negative_file ,encoding="utf8").readlines())
test_negative_examples = [s.strip() for s in test_negative_examples]
x_test = test_positive_examples + test_negative_examples
x_test = [clean_str(sent) for sent in x_test]
x_test = [s.split("|") for s in x_test]
# Generate labels

test_positive_labels = [1 for _ in test_positive_examples]
test_negative_labels = [0 for _ in test_negative_examples]
y_test = np.concatenate([test_positive_labels, test_negative_labels], 0)

In [ ]:
def my_tokenizer(s):
    x = [w.lower() for w in s]
    
    return x

In [ ]:
fstop = open('stopword.txt','r',encoding='utf-8')
stop_words = fstop.read().splitlines()
ngram_array = ((1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(3,3))
index= ['seg','model_variation','filter_sizes','precision','recall','f1','accuracy']
for ngram in ngram_array:
    count_vect = CountVectorizer(ngram_range=ngram,encoding='utf-8',lowercase=False, preprocessor=None, tokenizer=my_tokenizer)
    text_clf = Pipeline([('vect', count_vect),('tfidf', TfidfTransformer()),('clf', MLPClassifier())])
    text_clf.fit(x_text,y)
    predicted = text_clf.predict(x_test)
    print('gram: ',ngram,' : ',np.mean(predicted == y_test))
    precision = precision_score(y_test,predicted)
    recall = recall_score(y_test,predicted)
    f1 = f1_score(y_test,predicted)
    result = {}
    result['seg']=seg
    result['model_variation'] = 'MLP'
    result['filter_sizes'] = ngram
    result['precision']= precision
    result['recall']=recall
    result['f1']=f1
    result['accuracy']= np.mean(predicted == y_test) * 100
    print('\tPrecision:',precision)
    print('\tRecall:',recall)
    print('\tF1:',f1)
    
    df = pd.DataFrame(columns=index)
    df.loc[0]=result
    df.to_csv(directory+'Result-5/result_clickbait_MLP.csv', mode='a' , header=False,index=False)